In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
index_to_save = 9

<IPython.core.display.Javascript object>

In [4]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [5]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_ii",
    "Features": "Chemical + Mineralogical + Feature Engineering",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [6]:
df = pd.read_csv("../../../../../data/processed/partner_ii/Type-IL.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [7]:
df_copy = df.drop(
    [
        # Properties
        "loi",
        "water_demand",
        "setting_initial",
        "blaine",
        "sieve_32um",
        "sieve_45um",
        "CS1",
        "CS3",
        "CS7"
    ],
    axis=1,
).copy()


<IPython.core.display.Javascript object>

## Feature Engineering

In [8]:
# Feature Engineering over Chemical Features
ch_features = ["cao", "mgo", "na2o", "al2o3", "sio2", "so3", "k2o", "fe2o3"]

df_copy["std_ch_feats"] = df_copy[ch_features].std(ddof=0, axis=1)

df_copy["ratio_CaO_to_SiO2"] = df_copy["cao"] / df_copy["sio2"]
df_copy["ratio_MgO_to_CaO"] = df_copy["mgo"] / df_copy["cao"]

# Feature Engineering over Mineralogical Features
mi_features_set1 = ["aluminate", "ferrite"]
df_copy["std_mi_set1_feats"] = df_copy[mi_features_set1].std(ddof=0, axis=1)
df_copy["ratio_Aluminate_to_Ferrite"] = df_copy["aluminate"] / df_copy["ferrite"]

<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [9]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [10]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.737 (0.111)
MAE: -1.378 (0.106)
MAPE: -0.035 (0.003)
R2: 0.562 (0.051)


******
[TEST]
******
RMSE: -3.324 (1.626)
MAE: -2.332 (0.736)
MAPE: -0.058 (0.018)
R2: -1.118 (1.932)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   

  Cross Validation RMSE Test            MAE Test           MAPE Test  \
                        mean       std      mean       std      mean   
0   Repeated KFold  3.323912  1.625897  2.331785  0.736323  0.058391   

              R2 Test            
        std      mean       std  
0  0.018003 -1.118306  1.932285

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [12]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -0.000 (0.000)
MAE: -0.000 (0.000)
MAPE: -0.000 (0.000)
R2: 1.000 (0.000)


******
[TEST]
******
RMSE: -12.024 (11.640)
MAE: -9.361 (7.245)
MAPE: -0.237 (0.175)
R2: -379.569 (525.268)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  RMSE Test             MAE Test            \
                                    mean        std      mean       std   
0  Blocking Time Series Split  12.024184  11.639669  9.361049  7.244644   
1              Repeated KFold   3.323912   1.625897  2.331785  0.736323   

  MAPE Test               R2 Test              
       mean       std        mean         std  
0  0.237313  0.175115 -379.568708  525.267600  
1  0.058391  0.018003   -1.118306    1.932285

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [14]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.322 (0.705)
MAE: -1.035 (0.554)
MAPE: -0.026 (0.014)
R2: 0.616 (0.212)


******
[TEST]
******
RMSE: -8.462 (8.519)
MAE: -6.888 (6.534)
MAPE: -0.168 (0.149)
R2: -46.835 (65.320)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  RMSE Test             MAE Test            \
                                    mean        std      mean       std   
0  Blocking Time Series Split  12.024184  11.639669  9.361049  7.244644   
1              Repeated KFold   3.323912   1.625897  2.331785  0.736323   
2           Time Series Split   8.462288   8.518795  6.888300  6.534316   

  MAPE Test               R2 Test              
       mean       std        mean         std  
0  0.237313  0.175115 -379.568708  525.267600  
1  0.058391  0.018003   -1.118306    1.932285  
2  0.168271  0.148817  -46.834578   65.319840

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [16]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 1.901 (0.000)
MAE: 1.508 (0.000)
MAPE: 0.038 (0.000)
R2: 0.495 (0.000)


******
[TEST]
******
RMSE: 1.666 (0.000)
MAE: 1.302 (0.000)
MAPE: 0.034 (0.000)
R2: -0.662 (0.000)




<IPython.core.display.Javascript object>

In [17]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  RMSE Test             MAE Test            \
                                    mean        std      mean       std   
0  Blocking Time Series Split  12.024184  11.639669  9.361049  7.244644   
1           Out of time Split   1.665926   0.000000  1.302233  0.000000   
2              Repeated KFold   3.323912   1.625897  2.331785  0.736323   
3           Time Series Split   8.462288   8.518795  6.888300  6.534316   

  MAPE Test               R2 Test              
       mean       std        mean         std  
0  0.237313  0.175115 -379.568708  525.267600  
1  0.034464  0.000000   -0.661764    0.000000  
2  0.058391  0.018003   -1.118306    1.932285  
3  0.168271  0.148817  -46.834578   65.319840

<IPython.core.display.Javascript object>

In [18]:
coeff = pipeline.named_steps["estimator"].coef_

<IPython.core.display.Javascript object>

In [19]:
pipeline.named_steps["estimator"].intercept_

39.99935483870926

<IPython.core.display.Javascript object>

In [20]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [21]:
coeff = pipeline.named_steps["estimator"].coef_

<IPython.core.display.Javascript object>

In [22]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
std_mi_set1_feats,132.501466
aluminate,80.321229
sio2,40.736668
ratio_CaO_to_SiO2,34.677471
cao,26.812642
ratio_Aluminate_to_Ferrite,9.562398
ratio_MgO_to_CaO,7.060141
alite,2.726998
belite,1.914978
free_lime,0.746919


<IPython.core.display.Javascript object>

In [23]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params  RMSE Test             \
                                                          mean        std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  12.024184  11.639669   
1                                {"Test Size": 0.2}   1.665926   0.000000   
2                     {"N_Splits": 5, "Repeats": 3}   3.323912   1.625897   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   8.462288   8.518795   

   MAE Test           MAPE Test               R2 Test              
       mean       std      mean       std        mean         std  
0  9.361049  7.244644  0.237313  0.175115 -379.568708  525.267600  
1  1.302233  0.000000  0.034464  0.000000   -0.661764    0.000000  
2  2.331785  0.736323  0.058391  0.018003   -1.118306    1.932285  
3  6.888300  6.534316  0.168271  0.148817  -46.834578   65.319840

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [24]:
path = "../../../../../reports/results/local_models/partner_ii/type-IL/full/"
filename = f"linear_regression_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [25]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_ii/type-IL/grouped/"
filename = f"linear_regression_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>